<a href="https://colab.research.google.com/github/thoadao0301/AI-Project/blob/NamTP/Implement_Keras_Facenet_Vietnam_Famous_Celebrities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data processing**

---



## Mount data from drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Library

Requirement already satisfied: keras_facenet in /usr/local/lib/python3.7/dist-packages (0.3.2)

Requirement already satisfied: mtcnn in /usr/local/lib/python3.7/dist-packages (from keras_facenet) (0.1.0)

Requirement already satisfied: opencv-python>=4.1.0 in /usr/local/lib/python3.7/dist-packages (from mtcnn->keras_facenet) (4.1.2.30)

Requirement already satisfied: keras>=2.0.0 in /usr/local/lib/python3.7/dist-packages (from mtcnn->keras_facenet) (2.4.3)

Requirement already satisfied: numpy>=1.14.5 in /usr/local/lib/python3.7/dist-packages (from opencv-python>=4.1.0->mtcnn->keras_facenet) (1.19.5)

Requirement already satisfied: pyyaml in /usr/local/lib/python3.7/dist-packages (from keras>=2.0.0->mtcnn->keras_facenet) (3.13)

Requirement already satisfied: h5py in /usr/local/lib/python3.7/dist-packages (from keras>=2.0.0->mtcnn->keras_facenet) (2.10.0)

Requirement already satisfied: scipy>=0.14 in /usr/local/lib/python3.7/dist-packages (from keras>=2.0.0->mtcnn->keras_facenet) (1.4.1)

Requirement already satisfied: six in /usr/local/lib/python3.7/dist-packages (from h5py->keras>=2.0.0->mtcnn->keras_facenet) (1.15.0)


// Sklearn
Requirement already satisfied, skipping upgrade: joblib>=0.11 in /usr/local/lib/python3.7/dist-packages (from scikit-learn) (1.0.1)

Requirement already satisfied, skipping upgrade: scipy>=0.19.1 in /usr/local/lib/python3.7/dist-packages (from scikit-learn) (1.4.1)

Requirement already satisfied, skipping upgrade: numpy>=1.13.3 in /usr/local/lib/python3.7/dist-packages (from scikit-learn) (1.19.5)

Requirement already satisfied, skipping upgrade: threadpoolctl>=2.0.0 in /usr/local/lib/python3.7/dist-packages (from scikit-learn) (2.1.0)

In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 2.9MB/s 


In [ ]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [ ]:
!pip install keras_facenet

  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-cp37-none-any.whl size=10386 sha256=243b68b713c1c7ef95314389f992de51f5f5feda90a411fa21bea6446c89ba9c
  Stored in directory: /root/.cache/pip/wheels/f6/53/9a/36c4b52fd22faf4f710d5047d874655b85a1b2cf77accfb9bd
Successfully built keras-facenet


In [ ]:
import sys
import os, random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import cv2
import mtcnn
import pickle
from PIL import Image
from tqdm import tqdm
from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
from datetime import datetime
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## Remove images that has more than two faces or can't detect face

In [ ]:
def check_only_one_face(directory_path):
    list_sub_dir = os.listdir(directory_path)
    # create the detector, using default weights
    embedder = FaceNet()
    for sub_dir in list_sub_dir:
        count = 0
        sub_dir_path = os.path.join(directory_path, sub_dir)
        print(sub_dir,':')
        images = os.listdir(sub_dir_path)
        with tqdm(total=len(images), file=sys.stdout) as pbar:

            for image in images:
                image_path = os.path.join(sub_dir_path, image)
                # load image from file
                try:
                  img = Image.open(image_path)
                except:
                  try:
                    os.remove(image_path)
                    count+=1
                    continue
                  except:
                    print('Can\'t not delete file:', image_path)
                    continue

                # convert to RGB, if needed
                img = img.convert('RGB')
                # convert to array
                pixels = np.asarray(img)
                # detect faces in the image
                results = embedder.extract(pixels)
                if len(results) != 1:
                    try:
                      os.remove(image_path)
                      count+=1
                    except:
                      print('Can\'t not delete file:', image_path)
                pbar.update(1)
        print('DELETED: ', count)

In [ ]:
directory_path = '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male'
check_only_one_face(directory_path)

/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Pham_Hong_Phuoc
100%|██████████| 214/214 [03:56<00:00,  1.10s/it]
DELETED:  0
/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Ho_Trung_Dung
100%|██████████| 185/185 [03:11<00:00,  1.03s/it]
DELETED:  0
/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Ho_Viet_Trung
100%|██████████| 219/219 [03:17<00:00,  1.11it/s]
DELETED:  0
/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Ong_Cao_Thang
100%|██████████| 202/202 [03:38<00:00,  1.08s/it]
DELETED:  8
/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Luong_The_Thanh
100%|██████████| 203/203 [05:02<00:00,  1.49s/it]
DELETED:  30
/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Long_Nhat
100%|██████████| 256/256 [05:46<00:00,  1.35s/it]
DELETED:  48
/content/drive/Shareddrives/AI_Project/DatasetViet

In [ ]:
directory_path = '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Female'
check_only_one_face(directory_path)

## Create MTCNN face data

In [ ]:
def extract_face(filename,detector,required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    face_img = image.resize(required_size)

    return face_img

In [ ]:
def create_mtcnn_dataset(src,des,detector, overwrite=False):
    sub_directories = os.listdir(src)
    count = 1
    n = len(sub_directories)
    for sub_dir in sub_directories:

        sub_dir_src = os.path.join(src,sub_dir)
        sub_dir_des = os.path.join(des,sub_dir)

        try:
            os.mkdir(sub_dir_des)
        except:
            print('ERROR ON CREATING FOLDER', sub_dir_des)

        image_path_src = os.listdir(sub_dir_src)
        print('Loop:' , count,'/',n, ' ',sub_dir,':')

        with tqdm(total=len(image_path_src), file=sys.stdout) as pbar: 
            for image_path in image_path_src:

                image_path_src = os.path.join(sub_dir_src,image_path)
                image_path_des = os.path.join(sub_dir_des,image_path)

                check_exists = os.path.exists(image_path_des)
                if check_exists and overwrite:
                    face_img = extract_face(image_path_src,detector=detector)
                    face_img.save(image_path_des)
                    pbar.update(1)
                    continue
                
                if not check_exists:
                    face_img = extract_face(image_path_src,detector=detector)
                    face_img.save(image_path_des)
                pbar.update(1)

        count+=1

In [ ]:
detector = mtcnn.MTCNN()

In [ ]:
create_mtcnn_dataset('/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male',
                     '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/mtcnn_dataset_182_full',
                     detector, overwrite=False)

In [ ]:
create_mtcnn_dataset('/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Female',
                     '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/mtcnn_dataset_182_female_full',
                     embedder, overwrite=False)

# **Training classification with default weights**

---



## Prepare dataset

In [ ]:
def get_embedding(dir, embedder,use_MTCNN):
    x, imgs = [],[]
    for filename in listdir(dir):
        # set up file path
        filepath = os.path.join(dir,filename) 
        # load image from file
        image = Image.open(filepath)
        # convert to RGB, if needed
        image = image.convert('RGB')
        # convert to array
        pixels = np.asarray(image)
        # get embeddings for the faces in an image
        if use_MTCNN:
          pass
        else:
          detections = embedder.extract(pixels, threshold=0.95)
        if(len(detections)<1):
            print('ERROR: Can\'t not detect face: ',filepath)
            continue
        if(len(detections)>1):
            print('ERROR: More than two face detected: ', filepath)
            continue
        x.append(detections[0]['embedding'])
        imgs_path.append(filepath)
    return x,imgs_path


In [ ]:
def load_dataset(dir, embedder, use_MTCNN=False):
    x , y , imgs_path= [],[], []
    for subdir in listdir(dir):
        subdir_path = dir+'/'+subdir
        data_embeddings,data_paths = get_embedding(subdir_path, embedder,use_MTCNN)
        for i in range(len(data_embeddings)):
            x.append(data_embeddings[i])
            y.append(subdir)
            imgs_path.append(data_paths[i])
    return x, y, imgs_path

In [ ]:
# Create embedder tools with keras_facenet
embedder = FaceNet()

In [ ]:
# Create train dataset
dataset_train_path= '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male' # Train data path
x_dataset, y_dataset = load_dataset(dataset_train_path, embedder)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, test_size = 0.2, stratify=y_dataset)

## Training model classification using SVM method

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)

In [ ]:
# tranning model
model = SVC(kernel='linear', probability=True)
model.fit(train_x, train_y)
# score
score_train = model.score(train_x, train_y)
score_test = model.score(test_x, test_y)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

## Wrong prediction

In [ ]:
imgs_emd, labels,imgs_path  = load_dataset('/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male_Test',embedder)
X = np.array(X)
labels = np.array(labels)
imgs_emd = np.array(imgs_emd)

In [ ]:
def show_wrong_predict(imgs_path, labels, imgs_emd,model):
  labels_encode = out_encoder(labels)
  y_preds = model.predict(imgs_emd)
  idx_diff = np.flatnonzero(np.array(y_preds) != np.array(labels_encode))
  num_wrong_predict = len(idx_diff)
  print('Number of wrong predict in test set: ', )
  # display image and labels
  plt.figure(figsize=(15,15))
  
  for i in range(0,num_wrong_predic if num_wrong_predic < 10 else 10):
    plt.subplot(331+i)
    img = Image.open(imgs_path[idx_diff[i]])
    plt.imshow(img)
    plt.title(labels[idx_diff[i]] + '/ ' + out_encoder.inverse_transform(y_preds[idx_diff[i]])
  plt.show()

In [ ]:
show_wrong_predict(imgs_path,labels,imga_emd,model)

# **Try to increase accuracy by training the model FaceNet using triplet loss/softmax**

---



## **Using https://github.com/davidsandberg/facenet for create train dataset (Optional)** 

Code for implementing facenet by David Sandberg

### Install tensorflow version 1.* and scipy version 1.1.0

In [ ]:
!pip install -U --pre tensorflow-gpu=='1.*'

     |████████████████████████████████| 411.0MB 36kB/s 
     |████████████████████████████████| 3.8MB 36.0MB/s 
     |████████████████████████████████| 512kB 49.3MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 2.9MB 15.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=218706ec7d7d21b62220393b29565a22f495e917186aa9ce15a00a3528cb622e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have t

In [ ]:
!pip install scipy==1.1.0

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from drive.Shareddrives.AI_Project.facenet.src import facenet
import tensorflow as tf
import numpy as np
import argparse
import sys
import pickle
from scipy import misc

### Clone FaceNet model and set up enviroment

In [ ]:
%%bash
cd /content/drive/Shareddrives/AI_Project
git clone https://github.com/davidsandberg/facenet.git

Cloning into 'facenet'...


In [ ]:
%set_env PYTHONPATH=/content/drive/Shareddrives/AI_Project/facenet/src:/env/python

env: PYTHONPATH=/content/drive/Shareddrives/AI_Project/facenet/src:/env/python


### Create dataset with mtcnn

In [ ]:
# Code for fix bug
# replace line 85 with this for fix code - detect_face.py
# # save np.load
# np_load_old = np.load

# # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# # call load_data with allow_pickle implicitly set to true
# data_dict = np.load(data_path, encoding='latin1').item() #pylint: disable=no-member
# # restore np.load for future normal usage
# np.load = np_load_old

!python /content/drive/Shareddrives/AI_Project/facenet/src/align/align_dataset_mtcnn.py /content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female /content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/mtcnn_dataset_182_full --image_size 182 --margin 44

Streaming output truncated to the last 5000 lines.
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (205).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (206).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (207).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (208).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (209).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (21).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (210).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_full/VN_Female/Sam/Sam (211).jpg
/content/drive/Shareddrives/AI_Project/DataVietNam/mtcnn_dataset_182_female_fu

### Training model tensorflow 1 with tripletloss

In [ ]:
# Code for fix bug
# image = tf.to_float(image) # put code between line 118, and 119 in train_tripletloss.py

!python /content/drive/Shareddrives/AI_Project/facenet/src/train_tripletloss.py --logs_base_dir /content/drive/Shareddrives/AI_Project/facenet/logs/facenet/ --models_base_dir /content/drive/Shareddrives/AI_Project/model --data_dir /content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/DavidSandberg/mtcnn_dataset_182_full --image_size 160 --model_def models.inception_resnet_v1 --optimizer ADAM --learning_rate 0.0001 --weight_decay 1e-4 --max_nrof_epochs 500 --people_per_batch 21 --images_per_person 25

### Training model tensorflow 1 with softmax

In [ ]:
!python /content/drive/Shareddrives/AI_Project/facenet/src/train_softmax.py --logs_base_dir /content/drive/Shareddrives/AI_Project/facenet/logs/facenet/ --models_base_dir /content/drive/Shareddrives/AI_Project/model --data_dir /content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/DavidSandberg/mtcnn_dataset_182_full --image_size 160 --model_def models.inception_resnet_v1 --optimizer ADAM --learning_rate -1 --max_nrof_epochs 150 --keep_probability 0.8 --random_crop --random_flip --use_fixed_image_standardization --learning_rate_schedule_file /content/drive/Shareddrives/AI_Project/facenet/data/learning_rate_schedule_classifier_casia.txt --weight_decay 5e-4 --embedding_size 512 --lfw_distance_metric 1 --lfw_use_flipped_images --lfw_subtract_mean --validation_set_split_ratio 0.05 --validate_every_n_epochs 5 --prelogits_norm_loss_factor 5e-4

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Model directory: /content/drive/Shareddrives/AI_Project/model/20210525-084703
Log directory: /content/drive/Shareddrives/AI_Project/facenet/logs/facenet/20210525-084703

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.

### Train a classifier on own images

In [ ]:
!python classifier.py TRAIN C:\Users\namph\Downloads\Documents\Study\AI_Project\Dataset\mtcnn_dataset_182_full C:\Users\namph\Downloads\Documents\Study\David_Git\model\20180402-114759.pb C:\Users\namph\Downloads\Documents\Study\David_Git\model/lfw_classifier.pkl --batch_size 1000 --min_nrof_images_per_class 40 --nrof_train_images_per_class 35 --use_split_dataset

### Get embedding

In [ ]:
def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0/np.sqrt(np.prod(x.shape)))
    y = np.multiply(np.subtract(x, mean), 1/std_adj)
    return y  
def resize(image, image_size):
    if image.shape[1]>image_size:
        sz1 = int(image.shape[1]//2)
        sz2 = int(image_size//2)
        image = image[(sz1-sz2):(sz1+sz2),(sz1-sz2):(sz1+sz2),:]
    return image
def load_image(image_path, image_size, do_prewhiten=True):
    images = np.zeros((1, image_size, image_size, 3))
    img = Image.open(image_path)
    img = np.array(img.convert('RGB'))
    if do_prewhiten:
        img = prewhiten(img)
    img = resize(img, image_size)
    images[0,:,:,:] = img
    return images

In [ ]:
from drive.Shareddrives.AI_Project.facenet.src import facenet
def get_embedding(image_path,model_path,image_size=160):
        # Load the model
    with tf.Graph().as_default():
        with tf.compat.v1.Session() as sess:
            # Get input and output tensors
            print('Loading feature extraction model')
            facenet.load_model(model_path)
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            print('Calculating features for images')
            images = load_image(image_path, image_size)
            feed_dict = {images_placeholder: images, phase_train_placeholder: False}
            emb_array= sess.run(embeddings, feed_dict=feed_dict)
            return emb_array

### Load classfier model

In [ ]:
def load_model_classfier(model_path):
    with open(model_path, 'rb') as infile:
        (model, class_names) = pickle.load(infile)
        
    print('Loaded classifier model from file "%s"' % model_path)

    return model, class_names
    # predictions = model.predict_proba(emb_array)
    # best_class_indices = np.argmax(predictions, axis=1)
    # best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]

    # for i in range(len(best_class_indices)):
    #     print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))

    # accuracy = np.mean(np.equal(best_class_indices, labels))
    # print('Accuracy: %.3f' % accuracy)

    # OR

    # predictions = model.predict_proba(emb_array)
    # best_class_indices = np.argmax(predictions, axis=1)
    # best_class_probabilities = predictions[0,best_class_indices]
    # print('%4d  %s: %.3f' % (i, class_names[best_class_indices[0]], best_class_probabilities[0]))

In [ ]:
model, class_names = load_model_classfier('/content/drive/Shareddrives/AI_Project/DataVietNam/VietNam/Train_Checkpoint/lfw_classifier.pkl')

Loaded classifier model from file "/content/drive/Shareddrives/AI_Project/DataVietNam/VietNam/Train_Checkpoint/lfw_classifier.pkl"


## **Training keras_facenet model, tensorflow 2** ERROR, NOT FIX YET

### Create model, dataset

In [ ]:
# Create model FaceNet for training
model = FaceNet().model

In [ ]:
# Print model architecture
model.summary()

In [ ]:
# Create data from directory for training
def get_data_train(src):
    x , y = [],[]
    list_dir = os.listdir(src)
    for sub_dir in list_dir:
        list_image_path = os.path.join(src,sub_dir)
        list_image = os.listdir(list_image_path)

        with tqdm(total=len(list_image), file=sys.stdout) as pbar:
            for file_path in list_image:
                image_path = os.path.join(list_image_path, file_path)
                image = Image.open(image_path)
                # convert to RGB, if needed
                image = image.convert('RGB')
                # convert to array
                pixels = np.asarray(image)
                x.append(pixels)
                y.append(sub_dir)
                pbar.update(1)
    return np.array(x),np.array(y)

In [ ]:
# Create dataset for training
src = '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/mtcnn_dataset_182_full'
x_train_dataset, y_train_dataset = get_data_train(src)

In [ ]:
print(x_train_data.shape, len(y_train_data))

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_dataset, y_train_dataset, test_size=0.2,stratify = y_train_dataset)

### Install tensorflow-addons for using Triplet loss

In [ ]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 686kB 9.5MB/s 


### Training

In [ ]:
mcp = tf.keras.callbacks.ModelCheckpoint("/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/Train_Checkpoint/model_weights.h5",
                      save_best_only=True,save_weights_only=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(y_train)
y_train = out_encoder.transform(y_train)
y_valid = out_encoder.transform(y_valid)

In [ ]:
# Create data from original data
aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
# Create data shuffle with batch size
gen_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat().shuffle(1024).batch(128)

<BatchDataset shapes: ((None, 182, 182, 3), (None,)), types: (tf.uint8, tf.int64)>

In [ ]:
import tensorflow_addons as tfa

# Training model with TripletSemiHardLoss
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss(),
    metrics=["accuracy"])

In [ ]:
# Training model with binary_crossentropy
model.compile(loss="binary_crossentropy", 
              optimizer=tf.optimizers.Adam(0.001),
	            metrics=["accuracy"])

In [ ]:
#model.load_weights('/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/Train_Checkpoint/model_weights.h5')
# Training model
model.fit(aug.flow(x_train, y_train, batch_size=64),validation_data=(x_valid,y_valid), callbacks=[mcp],steps_per_epoch=x_train.shape[0]//64, epochs=100)

# **Retraining classification**

---



## Prepare dataset

In [ ]:
# Create embedder tools with keras_facenet
embedder = FaceNet()
embedder.model.load_weights()

In [ ]:
# Create train dataset
dataset_train_path= '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male' # Train data path
x_dataset, y_dataset = load_dataset(dataset_train_path, embedder)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, test_size = 0.2, stratify = y_dataset)

## Training model classification using SVM method

In [ ]:
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)

In [ ]:
# tranning model
model = SVC(kernel='linear', probability=True)
model.fit(train_x, train_y)
# score
score_train = model.score(train_x, train_y)
score_test = model.score(test_x, test_y)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

## Wrong prediction

In [ ]:
def prepare_test_data(data_dir,mtcnn):
  labels = []
  X = []
  imgs_emd = []
  for class_name in os.listdir(data_dir):
    image_path = os.path.join(data_dir,class_name)
    print(image_path)
    list_file = os.listdir(image_path)
    with tqdm(total=len(list_file), file=sys.stdout) as pbar:
      for image_file in list_file:
        imagefile = os.path.join(image_path,image_file)
        try:
          image = cv2.imread(imagefile)
          image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        except:
          print('ERROR: cant open')
          continue
        image = cv2.resize(image,(160,160))
        img_emd = embedder.extract(image, threshold=0.95)
        if(len(img_emd)<1):
            print('ERROR: Can\'t not detect face: ')
            continue
        if(len(img_emd)>1):
            print('ERROR: More than two face detected: ')
            continue
        imgs_emd.append(img_emd[0]['embedding'])
        X.append(image)
        labels.append(class_name)
        pbar.update(1)
  return X, labels,imgs_emd

In [ ]:
X_image, labels, imgs_emd = get_image_f_a_em('/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male_Test',mtcnn)
X = np.array(X)
labels = np.array(labels)
imgs_emd = np.array(imgs_emd)

In [ ]:
def show_wrong_predict(X_image, labels, imgs_emd,model):
  labels_encode = out_encoder(labels)
  y_preds = model.predict(imgs_emd)
  idx_diff = np.flatnonzero(np.array(y_preds) != np.array(labels_encode))
  num_wrong_predict = len(idx_diff)
  print('Number of wrong predict in test set: ', )
  # display image and labels
  plt.figure(figsize=(15,15))
  
  for i in range(0,num_wrong_predic if num_wrong_predic < 10 else 10):
    plt.subplot(331+i)
    plt.imshow(X_image_[idx_diff[i]])
    plt.title(labels[idx_diff[i]] + '/ ' + out_encoder.inverse_transform(y_preds[idx_diff[i]])
  plt.show()

In [ ]:
show_wrong_predict(X_image,labels,imga_emd,model)

## Save model

In [ ]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))

# **Prediction**

In [ ]:
def draw_bbox(image,bbox,text):
  x,y,w,h = bbox
  cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
  cv2.putText(image, text,(x, y), 
              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
  return image

In [ ]:
def out_video(images_folder,out_path,frame_size=15):
    img_array=[]
    list_file = os.listdir(images_folder)
    for i in range(len(list_file)):
        filename = f'{images_folder}/{i}.jpg'
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    
    
    out = cv2.VideoWriter(f'{out_path}/project.avi',cv2.VideoWriter_fourcc(*'DIVX'), frame_size, size)
    
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [ ]:
def out_txt(output_loc,duration,text):
    out_file = open(output_loc + '/list_faces.txt','w+')
    out_file.write(f'\nTime({duration}),{text}')
    out_file_txt.close()

In [ ]:
def detect_face(image,id,duration,out_file_txt,img_array=None):
  # convert to array
  pixels = np.asarray(image)
  # create the detector, using default weights
  detections = embedder.extract(pixels)
  # loop through each face in detections
  for detect in detections:
    # get embedded image
    embedded_img = detect['embedding']
    # get coordinates (x,y) and weight, height (w, h) of the bounding box
    bbox = detect['box']
    
    # expand dimension for feeding data to the model
    samples = np.expand_dims(embedded_img, axis=0)

    # get predict from model
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)

    # get label name
    class_index = yhat_class[0]
    predict_name = out_encoder.inverse_transform(yhat_class)[0]

    # get probability
    class_probability = round(yhat_prob[0,class_index] * 100)
    
    # Accept with the probability >50% 
    if id != -1:
        if class_probability > 50 and class_index == id:
          text = f'{predict_name}:{class_probability}'
          out_txt(out_file_txt,duration,text)
          if img_array is not None:
              image = draw_bbox(image,bbox,text)
              img_array.append(image)
        elif img_array is not None:
          img_array.append(image)     
    else:
        if class_probability > 50:
          text = f'{predict_name}:{class_probability}'
          out_txt(out_file_txt,duration,text)
          if img_array is not None:
              image = draw_bbox(image,bbox,text)
              img_array.append(image)
        elif img_array is not None:
          img_array.append(image) 

In [ ]:
from datetime import datetime
def get_loc(input_loc, output_loc,id=-1,export_video=False,frame_skip=0):
    
    now = datetime.now()
    dt_string = now.strftime("%d%m%Y%H%M%S")
    output_loc+=dt_string
    os.mkdir(output_loc)


    cap = cv2.VideoCapture(input_loc)

    fps = int(cap.get(cv2.CAP_PROP_FPS))  
    if frame_skip==0:
      frame_skip = fps
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number = 0
    img_array = None
    if export_video:
        img_array=[]

    print('Processing video...')
    with tqdm(total=frame_count//frame_skip-1, file=sys.stdout) as pbar:
        while cap.isOpened():
          # do stuff
          frame_number += frame_skip
          cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
          # Extract the frame
          ret, frame = cap.read()
          if ret == True:
              # Thực thi Face Detection
              duration = frame_number//fps
              detect_face(frame,id,duration,out_file_txt,img_array)
          else:
            break
          pbar.update(1)
    cap.release()
    print('Successful write .txt file')
    if export_video:
        print('Try to export .avi file')
        size = (img_array[0].shape[0],img_array[0].shape[1])
        print(size)
        out_path = output_loc + '/project.avi'
        print('Saving images...')
        images_path = output_loc + '/images'
        os.mkdir(images_path)
        video_images=[]
        with tqdm(total=len(img_array), file=sys.stdout) as pbar:
            for i in range(len(img_array)):
                cv2.imwrite(f'{images_path}/{i}.jpg',img_array[i])
                img = cv2.imread(f'{images_path}/{i}.jpg')
                video_images.append(img)
                pbar.update(1)
        print('Rendering video...')
        frame_size=fps//frame_skip
        out_video(images_path,output_loc,frame_size)
        print('Successful export .avi file')
    

In [ ]:
cap = cv2.VideoCapture(input_loc)

fps = int(cap.get(cv2.CAP_PROP_FPS))  
cap.release()
out_video('/content/drive/Shareddrives/AI_Project/Test/24052021160209/images','/content/drive/Shareddrives/AI_Project/Test/24052021160209',fps)

In [ ]:
input_loc = '/content/drive/Shareddrives/AI_Project/testvid.mp4'
output_loc = '/content/drive/Shareddrives/AI_Project/Test/'

In [ ]:
def finding_fast(input_loc,output_loc,export_video=False):
    get_loc(input_loc, output_loc,export_video=export_video,frame_skip=0)

In [ ]:
finding_fast(input_loc,output_loc,export_video=True)

Processing video...
100%|██████████| 59/59 [01:21<00:00,  1.38s/it]
Successful write .txt file
Try to export .avi file
(1080, 1920)
Saving images...
100%|██████████| 102/102 [00:07<00:00, 13.66it/s]
Rendering video...
1
Successful export .avi file


In [ ]:
def finding_quality(input_loc,output_loc,export_video=False):
    get_loc(input_loc, output_loc,export_video=export_video,frame_skip=1)

In [ ]:
test_img = cv2.imread('/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male/Ali_HoangDuong/Ali_HoangDuong (12).jpg')
type(test_img)

In [ ]:
finding_quality(input_loc,output_loc,export_video=True)

Processing video...
100%|█████████▉| 1498/1501 [33:26<00:04,  1.34s/it]
Successful write .txt file
Try to export .avi file
2537it [00:00, 294705.99it/s]           
Successful export .avi file


In [ ]:
for id in range(100):
    labels_id = out_encoder.inverse_transform([id])
    print(f'{id} : {labels_id}')

0 : ['Ali_HoangDuong']
1 : ['BD_Tran']
2 : ['Bao_Lam']
3 : ['Binz']
4 : ['Bui_Anh_Tuan']
5 : ['Chau_Gia_Kiet']
6 : ['Chau_Khai_Phong']
7 : ['Chi_Dan']
8 : ['Chi_Thien']
9 : ['Dai_Nhan']
10 : ['Dam_Vinh_Hung']
11 : ['Dan_Nguyen']
12 : ['Dan_Truong']
13 : ['Dao_Ba_Loc']
14 : ['Don_Nguyen']
15 : ['Duong_Khac_Linh']
16 : ['Duy_Khanh']
17 : ['Gin_Tuan_Kiet']
18 : ['Ha_Anh_Tuan']
19 : ['Hamtet_Truong']
20 : ['Ho_Quang_Hieu']
21 : ['Ho_Trung_Dung']
22 : ['Ho_Viet_Trung']
23 : ['Hoai_Lam']
24 : ['Hoai_Linh']
25 : ['Hoang_Rapper']
26 : ['Huy_Tran']
27 : ['Huynh_Lap']
28 : ['Huynh_Phuong']
29 : ['Isaac']
30 : ['Justatee']
31 : ['Kenvin_Khanh']
32 : ['Khac_Viet']
33 : ['Khanh_Phuong']
34 : ['Khuong_Ngoc']
35 : ['Kieu_Minh_tuan']
36 : ['Kim_Ly']
37 : ['Lam_Hung']
38 : ['Lam_Truong']
39 : ['Lam_Vinh_Hai']
40 : ['Long_Nhat']
41 : ['Lou_Hoang']
42 : ['Luong_Bang_Quang']
43 : ['Luong_Manh_Hai']
44 : ['Luong_The_Thanh']
45 : ['MC_Thanh_Trung']
46 : ['Mai_Tai_Phen']
47 : ['Minh_Du']
48 : ['Minh_Luan']
4

In [ ]:
def finding_custom(input_loc,output_loc,id,frame_skip=0,export_video=False):
    get_loc(input_loc, output_loc,id=id,export_video=export_video,frame_skip=frame_skip)

In [ ]:
finding_custom(input_loc,output_loc,86,frame_skip=3,export_video=True)

Processing video...
100%|██████████| 499/499 [11:49<00:00,  1.42s/it]
Successful write .txt file
Try to export .avi file
(1080, 1920)
Saving images...
100%|██████████| 848/848 [01:06<00:00, 12.66it/s]
Rendering video...


NameError: ignored

# **DEMO**

## Library

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from drive.Shareddrives.AI_Project.facenet.src import facenet
from datetime import datetime

## Extract faces data

In [ ]:
def extract_faces(img_array,detector,required_size=(160, 160)):
    faces_list,bbox = [],[]
    # load image from file
    image = Image.fromarray(img_array)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    for face in results:
        x1, y1, width, height = face['box']
        
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        face_img = image.resize(required_size)
        faces_list.append(face_img)
        bbox.append(face['box'])

    return faces_list,bbox

## Get embedding

In [ ]:
def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0/np.sqrt(np.prod(x.shape)))
    y = np.multiply(np.subtract(x, mean), 1/std_adj)
    return y  
def resize(image, image_size):
    if image.shape[1]>image_size:
        sz1 = int(image.shape[1]//2)
        sz2 = int(image_size//2)
        image = image[(sz1-sz2):(sz1+sz2),(sz1-sz2):(sz1+sz2),:]
    return image
def load_image(image_path, image_size, do_prewhiten=True):
    images = np.zeros((1, image_size, image_size, 3))
    img = Image.fromarray(image_path)
    img = np.array(img.convert('RGB'))
    if do_prewhiten:
        img = prewhiten(img)
    img = resize(img, image_size)
    images[0,:,:,:] = img
    return images

In [ ]:
def get_embedding(image_path,image_size=160):
    # Get input and output tensors
    images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
    embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
    phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
    print('Calculating features for images')
    images = load_image(image_path, image_size)
    feed_dict = {images_placeholder: images, phase_train_placeholder: False}
    emb_array= sess.run(embeddings, feed_dict=feed_dict)
    return emb_array

## Load classfier model

In [ ]:
def load_model_classfier(model_path):
    with open(model_path, 'rb') as infile:
        (model, class_names) = pickle.load(infile)
        
    print('Loaded classifier model from file "%s"' % model_path)

    return model, class_names

## Prediction

In [ ]:
def draw_bbox(image,bbox,text):
  x,y,w,h = bbox
  cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
  cv2.putText(image, text,(x, y), 
              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
  return image

In [ ]:
def out_video(images_folder,out_path,frame_size=15):
    img_array=[]
    list_file = os.listdir(images_folder)
    for i in range(len(list_file)):
        filename = f'{images_folder}/{i}.jpg'
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    
    
    out = cv2.VideoWriter(f'{out_path}/project.avi',cv2.VideoWriter_fourcc(*'DIVX'), frame_size, size)
    
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [ ]:
def out_txt(output_loc,duration,text):
    out_file = open(output_loc + '/list_faces.txt','w+')
    out_file.write(f'\nTime({duration}),{text}')
    out_file_txt.close()

In [ ]:
def detect_face(image,id,duration,out_file_txt,detector,img_array=None,threshold=0.5):
  # create the detector, using default weights
  faces_list,bbox = extract_faces(image,detector,required_size=(160, 160))
  # loop through each face in detections
  for i in range(len(faces_list):
    # get embedded image
    embedded_img = get_embedding(faces_list[i],image_size=160)
    # get coordinates (x,y) and weight, height (w, h) of the bounding box
    bbox = bbox[i]
    
    # expand dimension for feeding data to the model
    samples = np.expand_dims(embedded_img, axis=0)

    # get predict from model
    predictions = model.predict_proba(samples)
    class_index = np.argmax(predictions, axis=1)
    
    # get label name
    predict_name = class_names[class_index[0]]

    # get probability
    class_probability = round(predictions[0,class_index] * 100)
    threshold*=100
    # Accept with the probability > threshold
    if id != -1:
        if class_probability > threshold and class_index == id:
          text = f'{predict_name}:{class_probability}'
          out_txt(out_file_txt,duration,text)
          if img_array is not None:
              image = draw_bbox(image,bbox,text)
              img_array.append(image)
        elif img_array is not None:
          img_array.append(image)     
    else:
        if class_probability > threshold:
          text = f'{predict_name}:{class_probability}'
          out_txt(out_file_txt,duration,text)
          if img_array is not None:
              image = draw_bbox(image,bbox,text)
              img_array.append(image)
        elif img_array is not None:
          img_array.append(image) 

List id:

0 : ['Ali_HoangDuong']

1 : ['BD_Tran']

2 : ['Bao_Lam']

3 : ['Binz']

4 : ['Bui_Anh_Tuan']

5 : ['Chau_Gia_Kiet']

6 : ['Chau_Khai_Phong']

7 : ['Chi_Dan']

8 : ['Chi_Thien']

9 : ['Dai_Nhan']

10 : ['Dam_Vinh_Hung']

11 : ['Dan_Nguyen']

12 : ['Dan_Truong']

13 : ['Dao_Ba_Loc']

14 : ['Don_Nguyen']

15 : ['Duong_Khac_Linh']

16 : ['Duy_Khanh']

17 : ['Gin_Tuan_Kiet']

18 : ['Ha_Anh_Tuan']

19 : ['Hamtet_Truong']

20 : ['Ho_Quang_Hieu']

21 : ['Ho_Trung_Dung']

22 : ['Ho_Viet_Trung']

23 : ['Hoai_Lam']

24 : ['Hoai_Linh']

25 : ['Hoang_Rapper']

26 : ['Huy_Tran']

27 : ['Huynh_Lap']

28 : ['Huynh_Phuong']

29 : ['Isaac']

30 : ['Justatee']

31 : ['Kenvin_Khanh']

32 : ['Khac_Viet']

33 : ['Khanh_Phuong']

34 : ['Khuong_Ngoc']

35 : ['Kieu_Minh_tuan']

36 : ['Kim_Ly']

37 : ['Lam_Hung']

38 : ['Lam_Truong']

39 : ['Lam_Vinh_Hai']

40 : ['Long_Nhat']

41 : ['Lou_Hoang']

42 : ['Luong_Bang_Quang']

43 : ['Luong_Manh_Hai']

44 : ['Luong_The_Thanh']

45 : ['MC_Thanh_Trung']

46 : ['Mai_Tai_Phen']

47 : ['Minh_Du']

48 : ['Minh_Luan']

49 : ['Nam_Cuong']

50 : ['Ngo_Kien_Huy']

51 : ['Nguyen_Khang']

52 : ['Nguyen_Phi_Hung']

53 : ['Nguyen_Tran_Trung_Quan']

54 : ['Nhan_Phuc_Vinh']

55 : ['Noo_Phuoc_Thinh']

56 : ['Ong_Cao_Thang']

57 : ['OnlyC']

58 : ['Pham_Hong_Phuoc']

59 : ['Pham_Truong']

60 : ['Phan_Anh']

61 : ['Phan_Manh_Quynh']

62 : ['Quang_Ha']

63 : ['Quang_Le']

64 : ['Quang_Trung_Tran']

65 : ['Quy_Binh']

66 : ['Quyen_Linh']

67 : ['Rocker_Nguyen']

68 : ['ST']

69 : ['Son_Tung']

70 : ['Soobin_HoangSon']

71 : ['TIM']

72 : ['Ta_Quang_Thang']

73 : ['Thai_Vu']

74 : ['Thanh_Bach']

75 : ['Thanh_Duy']

76 : ['Thanh_Loc']

77 : ['Tien_Luat']

78 : ['Touliver']

79 : ['Tran_Thanh']

80 : ['Tran_Tien_Dat']

81 : ['Trinh_Dinh_Quang']

82 : ['Trinh_Thang_Binh']

83 : ['Trong_Hieu']

84 : ['Truc_Nhan']

85 : ['Trung_Quan']

86 : ['Truong_Giang']

87 : ['Truong_Nam_Thanh']

88 : ['Truong_The_Vinh']

89 : ['Truong_Vu']

90 : ['Tu_Long']

91 : ['Tuan_Hung']

92 : ['Tung_Duong']

93 : ['Ung_Dai_Ve']

94 : ['Ung_Hoang_Phuc']

95 : ['Vinh_Rau']

96 : ['Will']

97 : ['Xuan_Bac']

98 : ['Xuan_Nghi']

99 : ['chubin']

## Main

In [ ]:
input_loc = '/content/drive/Shareddrives/AI_Project/testvid.mp4'
output_loc = '/content/drive/Shareddrives/AI_Project/Test/'
#input_loc, output_loc,id=id,export_video=export_video,frame_skip=frame_skip

In [ ]:
#@title Parameter

input_loc = 'input_loc' #@param {type:"string"}
output_loc = 'output_loc' #@param {type:"string"}
model_path = 'model_path' #@param {type:"string"}
export_video = True #@param ["False", "True"] {type:"raw"}
frame_skip = 0 #@param {type:"slider", min:0, max:60, step:1}
threshold = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}

#main
with tf.Graph().as_default():
    with tf.Session() as sess:
        # Load facenet model
        print('Loading feature extraction model')
        facenet.load_model(model_path)

        # Create mtcnn model
        detector = mtcnn.MTCNN()
        # Create folder
        now = datetime.now()
        dt_string = now.strftime("%d%m%Y%H%M%S")
        output_loc+=dt_string
        os.mkdir(output_loc)

        # Read video
        cap = cv2.VideoCapture(input_loc)

        # Get fpt from video
        fps = int(cap.get(cv2.CAP_PROP_FPS))  

        # Get frame information
        if frame_skip==0:
          frame_skip = fps
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_number = 0

        # Create array for export video
        img_array = None
        if export_video:
            img_array=[]

        print('Processing video...')
        with tqdm(total=frame_count//frame_skip, file=sys.stdout) as pbar:
            while cap.isOpened():
              # Skip frame
              frame_number += frame_skip
              cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

              # Extract the frame
              ret, frame = cap.read()

              if ret == True:
                  # Get frame position in second
                  duration = frame_number//fps
                  # Face Detection
                  detect_face(frame,id,duration,out_file_txt,
                              detector, img_array, threshold)
              else:
                break
              pbar.update(1)
        cap.release()
        print('Successful write .txt file')

        # Export video
        if export_video:
            print('Try to export .avi file')
            # Get image size
            size = (img_array[0].shape[0],img_array[0].shape[1])
            
            print('Saving images...')
            images_path = output_loc + '/images'
            os.mkdir(images_path)

            
            video_images=[]
            with tqdm(total=len(img_array), file=sys.stdout) as pbar:
                for i in range(len(img_array)):
                    cv2.imwrite(f'{images_path}/{i}.jpg',img_array[i])
                    img = cv2.imread(f'{images_path}/{i}.jpg')
                    video_images.append(img)
                    pbar.update(1)
            print('Rendering video...')
            frame_size=fps//frame_skip
            out_path = output_loc + '/project.avi'
            out_video(images_path,out_path,frame_size)
            print('Successful export .avi file')

# **VGG-Face**

In [ ]:
pip install keras-vggface

In [ ]:
pip install mtcnn

In [ ]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

In [ ]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

In [ ]:
# Example of face detection with a vggface2 model 
# load the photo and extract the face
pixels = extract_face('sharon_stone1.jpg')
# convert one face into samples
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(samples, version=2)
# create a vggface model
model = VGGFace(model='resnet50')
# perform prediction
yhat = model.predict(samples)
# convert prediction into names
results = decode_predictions(yhat)
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

In [ ]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)
	return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))

In [ ]:
# face verification with the VGGFace2 model
# define filenames
filenames = ['sharon_stone1.jpg', 'sharon_stone2.jpg',
	'sharon_stone3.jpg', 'channing_tatum.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# define sharon stone
sharon_id = embeddings[0]
# verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[1])
is_match(embeddings[0], embeddings[2])
# verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[3])

# **OpenFace**

https://github.com/iwantooxxoox/Keras-OpenFace

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

In [ ]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

In [ ]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

In [ ]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [ ]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name.split('_')[0], euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.8:
        return str(name)
    else:
        return None